In [2]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
import dgl
import torch as th

from sklearn.preprocessing import LabelEncoder

### Préparation des interactions Client - Article

- Extraction de N identifiants utilisateurs 
- Split entre le jeu d'entraînement et de test
- Compilation des features

In [63]:
transactions = pd.read_pickle('pickles/transactions.pkl')

In [66]:
transactions_sampled = transactions.sample(frac = 0.1)
transactions_sampled.to_pickle('pickles/transactions_sampled.pkl')

In [3]:
customers = pd.read_pickle('pickles/customers_second_iteration.pkl')
articles = pd.read_pickle('pickles/articles_second_iteration.pkl')

In [4]:
customers['fashion_news_frequency'].replace({'None': 'NONE'}, inplace = True)

customers.drop(columns = ['baby', 'divided', 'ladieswear', 'menswear', 'age', 'average_cart_articles',
       'average_cart_price', 'total_carts', 'total_articles', 'total_price', 'sport', 'repurchases'], axis = 1, inplace = True)

# Typer les champs de catégorie
customers[['fashion_news_frequency','club_member_status']] = customers[['fashion_news_frequency','club_member_status']].astype('category')

In [5]:
from utils.dummify import Dummify

dummify = Dummify()

customers_full = dummify.transform(customers)
# Nan values for average_cart_interval / repurchases_interval
customers_full = customers_full.fillna(0)


In [8]:
customers_full.to_pickle('pickles/customers_gnn_full.pkl')
customers_full.head()

,customer_id,FN,Active,average_cart_interval,repurchases_interval,age_around_15,age_around_25,age_around_35,age_around_45,age_around_55,age_around_65,postal_code_group,group,club_member_status:ACTIVE,club_member_status:LEFT CLUB,club_member_status:NON ACTIVE,club_member_status:PRE-CREATE,fashion_news_frequency:Monthly,fashion_news_frequency:NONE,fashion_news_frequency:Regularly
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0,0,9.809524,0.000000,0.0,0.0,0.0,0.6,0.4,0.0,4,7,1,0,0,0,0,1,0
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0,0,4.259740,5.285714,0.0,1.0,0.0,0.0,0.0,0.0,4,1,1,0,0,0,0,1,0
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0,0,17.285714,0.000000,0.1,0.9,0.0,0.0,0.0,0.0,4,1,1,0,0,0,0,1,0
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0,0,0.000000,0.000000,0.0,0.0,0.0,0.1,0.9,0.0,1,4,1,0,0,0,0,1,0
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1,1,19.142857,2.000000,0.0,0.0,0.0,0.3,0.7,0.0,1,3,1,0,0,0,0,0,1


In [10]:
from utils.dummify import Dummify
categories = ['product_type_name', 'product_group_name',
       'graphical_appearance_name', 'colour_group_name',
       'perceived_colour_value_name', 'perceived_colour_master_name',
       'department_name', 'index_name', 'index_group_name', 'section_name',
       'garment_group_name']

columns_to_drop = ['total_purchases', 'average_quantity',
       'average_price', 'age_around_15', 'age_around_25',
       'age_around_35', 'age_around_45', 'age_around_55', 'age_around_65',
       'repurchases', 'repurchase_interval']

articles.drop(columns = columns_to_drop, axis = 1, inplace = True)

articles['has_image'] = articles['has_image'].astype(int)

articles[categories] = articles[categories].astype('category')

dummify = Dummify()

articles_full = dummify.transform(articles)


In [13]:
articles_full.to_pickle('pickles/articles_gnn_full.pkl')
articles_full.head(1)

,article_id,has_image,product_type_name:Accessories set,product_type_name:Alice band,product_type_name:Baby Bib,product_type_name:Backpack,product_type_name:Bag,product_type_name:Ballerinas,product_type_name:Beanie,product_type_name:Belt,...,garment_group_name:Shorts,garment_group_name:Skirts,garment_group_name:Socks and Tights,garment_group_name:Special Offers,garment_group_name:Swimwear,garment_group_name:Trousers,garment_group_name:Trousers Denim,"garment_group_name:Under-, Nightwear",garment_group_name:Unknown,garment_group_name:Woven/Jersey/Knitted mix Baby
0,0108775015,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [60]:
to_drop_in_medium =  ['product_type_name',
       'graphical_appearance_name', 'colour_group_name',
       'department_name', ]

articles_medium = articles.drop(columns = to_drop_in_medium, axis = 1)

articles_medium = dummify.transform(articles_medium)


In [62]:
articles_medium.to_pickle('pickles/articles_gnn_medium.pkl')
articles_medium.head(1)

,article_id,has_image,product_group_name:Accessories,product_group_name:Bags,product_group_name:Cosmetic,product_group_name:Fun,product_group_name:Furniture,product_group_name:Garment Full body,product_group_name:Garment Lower body,product_group_name:Garment Upper body,...,garment_group_name:Shorts,garment_group_name:Skirts,garment_group_name:Socks and Tights,garment_group_name:Special Offers,garment_group_name:Swimwear,garment_group_name:Trousers,garment_group_name:Trousers Denim,"garment_group_name:Under-, Nightwear",garment_group_name:Unknown,garment_group_name:Woven/Jersey/Knitted mix Baby
0,0108775015,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
articles_medium.to_pickle('pickles/articles_gnn_medium.pkl')

In [2]:
import pandas as pd
articles = pd.read_pickle('pickles/articles_gnn_full.pkl')
customers = pd.read_pickle('pickles/customers_gnn_full.pkl')

In [4]:
customers.isna().sum()

customer_id                              0
FN                                       0
Active                                   0
average_cart_interval               457672
repurchases_interval                  9699
age_around_15                            0
age_around_25                            0
age_around_35                            0
age_around_45                            0
age_around_55                            0
age_around_65                            0
postal_code_group                        0
group                                    0
club_member_status:ACTIVE                0
club_member_status:LEFT CLUB             0
club_member_status:NON ACTIVE            0
club_member_status:PRE-CREATE            0
fashion_news_frequency:Monthly           0
fashion_news_frequency:NONE              0
fashion_news_frequency:Regularly         0
dtype: int64